# Distance between heatmaps and Ekman GT masks

First thresholds the heatmaps representing important regions of the face for every model and expression using Otsu's method, and then computes the distance with the Ekman GT masks. 
- Several distances were used: IOU, Precision, Recall, and F1 score.

In [2]:
import os
import cv2

from ekman_expressions.heatmaps import apply_threshold, get_masks_distance, plot_bar

In [3]:
# Models to evaluate
model_names = ['SilNet', 'WeiNet', 'AlexNet', 'SongNet', 'InceptionV3',
               'VGG19', 'VGG16', 'ResNet50', 'ResNet101V2', 'Xception',
               'MobileNetV3Large', 'EfficientNetV2B0']

# Number of k-cross validations and folder where they are located
# Alternatively set the paths to the target training and test manually
K = 5
dataset_paths_root = '../datasets/'
dataset_paths_train = []
dataset_paths_test = []
for i in range(K):
    dataset_paths_train.append(dataset_paths_root + 'CV' + str(i+1))
    dataset_paths_test.append(dataset_paths_root + 'CV' + str(i+1) + '_test')

# Folders where the heatmaps are saved
heatmaps_dir_gray = '../heatmaps/GRAY'
heatmaps_dir_th = '../heatmaps/THRESHOLD'
heatmaps_gt_dir = '../heatmaps/GROUND_TRUTH'

# Folders where to store barplots
barplots_dir = '../barplots/th_otsu'

# Labels of the classes
label_names = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']

## Threshold the heatmaps using OTSU

In [21]:
# Subdirectories of heatmaps
heatmap_th_paths = [os.path.join(heatmaps_dir_th, 'heatmaps_by_cv'), os.path.join(heatmaps_dir_th, 'heatmaps_by_net'), os.path.join(heatmaps_dir_th, 'heatmaps_total')]
heatmap_gray_paths = [os.path.join(heatmaps_dir_gray, 'heatmaps_by_cv'), os.path.join(heatmaps_dir_gray, 'heatmaps_by_net'), os.path.join(heatmaps_dir_gray, 'heatmaps_total')]

# Create threshold heatmaps folders if they don't exist
for f in heatmap_th_paths:
    if not os.path.exists(f):
        os.mkdir(f)

# Apply threshold to every heatmap
for gray_path, th_path in zip(heatmap_gray_paths, heatmap_th_paths):
    for heatmap_name in os.listdir(gray_path):
        heatmap = cv2.imread(os.path.join(gray_path, heatmap_name), 0)
        th = apply_threshold(heatmap, th='otsu')
        cv2.imwrite(os.path.join(th_path, heatmap_name), th)

## Calculate difference between heatmaps and ground truth

In [27]:
# Subdirectories of heatmaps
heatmaps_cv_path = os.path.join(heatmaps_dir_th, 'heatmaps_by_cv')
heatmaps_model_path = os.path.join(heatmaps_dir_th, 'heatmaps_by_net')
heatmaps_total_path = os.path.join(heatmaps_dir_th, 'heatmaps_total')

# Name and distance for each heatmap
cv_names_list = [[] for _ in range(len(label_names))]
cv_dists_list = [[] for _ in range(len(label_names))]
net_names_list = [[] for _ in range(len(label_names))]
net_dists_list = [[] for _ in range(len(label_names))]
exp_names_list = []
exp_dists_list = []
total_names_list = []
total_dists_list = []

# Load ground truth heatmaps
gt_heatmaps = []
for expression in label_names:
    gt_heatmaps.append(cv2.imread(os.path.join(heatmaps_gt_dir, expression + '_gt.png'), 0) / 255)

method = 'iou' # Also 'precision', 'recall', 'f1_score'
    
for class_i, expression in enumerate(label_names):
    
    for model_name in model_names:
        
        for train_path in dataset_paths_train:
            
            # By CV subsets
            # Load heatmap and compute distance
            h_name = model_name + '_' + os.path.basename(train_path) + '_' + expression + '_heatmap.png'
            heatmap = cv2.imread(os.path.join(heatmaps_cv_path, h_name), 0) / 255
            dist = get_masks_distance(gt_heatmaps[class_i], heatmap, method=method)
            
            # Store name and distance
            cv_names_list[class_i].append(model_name + '_' + os.path.basename(train_path))
            cv_dists_list[class_i].append(dist)
        
        # By net
        # Load heatmap and compute distance
        h_name = model_name + '_' + expression + '_heatmap.png'
        heatmap = cv2.imread(os.path.join(heatmaps_model_path, h_name), 0) / 255
        dist = get_masks_distance(gt_heatmaps[class_i], heatmap, method=method)

        # Store name and distance
        net_names_list[class_i].append(model_name)
        net_dists_list[class_i].append(dist)

        # Store name and distance
        exp_names_list.append(expression + '_' + model_name)
        exp_dists_list.append(dist)

    # Total
    # Load heatmap and compute distance
    h_name = expression + '_heatmap.png'
    heatmap = cv2.imread(os.path.join(heatmaps_total_path, h_name), 0) / 255
    dist = get_masks_distance(gt_heatmaps[class_i], heatmap, method=method)

    # Store name and distance
    total_names_list.append(expression)
    total_dists_list.append(dist)

## Show results

### Plot sorted barplots

In [ ]:
for class_i, expression in enumerate(label_names):
    
    # By net
    plot_bar(
        labels=net_names_list[class_i],
        distances=net_dists_list[class_i],
        title=expression + ' by net',
        save_path=os.path.join(barplots_dir, expression + '_by_net'),
        sort=True, 
        width=15, 
        height=10, 
        title_size=25, 
        font_size=16,
        reverse=True
    )
    
    # By CV set
    plot_bar(
        labels=cv_names_list[class_i],
        distances=cv_dists_list[class_i],
        title=expression + ' by CV set',
        save_path=os.path.join(barplots_dir, expression + '_by_CV'),
        sort=True, 
        width=30, 
        height=35, 
        title_size=40, 
        font_size=16,
        reverse=True
    )

# By expression
plot_bar(
    labels=total_names_list,
    distances=total_dists_list,
    title='By expression',
    save_path=os.path.join(barplots_dir, 'By expression'),
    sort=True, 
    width=8, 
    height=5, 
    title_size=16, 
    font_size=12,
    reverse=True
)

### Print sorted results

In [23]:
print('BY EXPRESSION:')
for net_name, net_dist in sorted(list(zip(total_names_list, total_dists_list)), key=lambda t: t[1], reverse=True):
    print('\t'+net_name + ':', net_dist)

print('\nBY NETWORK:')
for class_i, expression in enumerate(label_names):
    print('\t'+expression.upper()+':')
    for net_name, net_dist in sorted(list(zip(net_names_list[class_i], net_dists_list[class_i])), key=lambda t: t[1], reverse=True):
        print('\t\t'+net_name + ':', net_dist)

print('\nBY CV:')
for class_i, expression in enumerate(label_names):
    print('\t'+expression.upper()+':')
    for net_name, net_dist in sorted(list(zip(cv_names_list[class_i], cv_dists_list[class_i])), key=lambda t: t[1], reverse=True):
        print('\t\t'+net_name + ':', net_dist)

BY EXPRESSION:
	Disgust: 0.6439009060912376
	Anger: 0.3488223203153958
	Sadness: 0.2983485593815882
	Surprise: 0.26208947062643106
	Happiness: 0.20512300278975398
	Fear: 0.189915575865541

BY NETWORK:
	ANGER:
		SongNet: 0.35198259741251003
		MobileNetV3Large: 0.33223505698116135
		AlexNet: 0.3311141304347826
		InceptionV3: 0.30730469324702075
		SilNet: 0.24431602879878742
		EfficientNetV2B0: 0.24011563988761758
		WeiNet: 0.2148104511747176
		ResNet50: 0.19436593178820427
		ResNet101V2: 0.181093087433533
		Xception: 0.1544771018455229
		VGG19: 0.12403793000807926
		VGG16: 0.11635888217028927
	DISGUST:
		EfficientNetV2B0: 0.7211777452042573
		InceptionV3: 0.6411534519705581
		VGG19: 0.632024592816309
		VGG16: 0.5751513029560346
		ResNet101V2: 0.5697335148803735
		MobileNetV3Large: 0.5249689954526664
		ResNet50: 0.5156376019356573
		Xception: 0.4138696617474207
		SilNet: 0.3827861433390932
		WeiNet: 0.3351951896956933
		SongNet: 0.23384494293585203
		AlexNet: 0.18727499307671006
	FEAR:
		

### Print non sorted results

In [9]:
print('BY EXPRESSION:')
for net_name, net_dist in zip(total_names_list, total_dists_list):
    print('\t'+net_name + ':', net_dist)

print('\nBY NETWORK:')
for class_i, expression in enumerate(label_names):
    print('\t'+expression.upper()+':')
    for net_name, net_dist in zip(net_names_list[class_i], net_dists_list[class_i]):
        print('\t\t'+net_name + ':', net_dist)

print('\nBY CV:')
for class_i, expression in enumerate(label_names):
    print('\t'+expression.upper()+':')
    for net_name, net_dist in zip(cv_names_list[class_i], cv_dists_list[class_i]):
        print('\t\t'+net_name + ':', net_dist)

BY EXPRESSION:
	Anger: 0.16805194805194806
	Disgust: 0.19957792207792208
	Fear: 0.0796103896103896
	Happiness: 0.06564935064935065
	Sadness: 0.1378409090909091
	Surprise: 0.12449675324675324

BY NETWORK:
	ANGER:
		SilNet: 0.12560064935064935
		WeiNet: 0.10063311688311688
		AlexNet: 0.15824675324675325
		SongNet: 0.14972402597402598
		InceptionV3: 0.13563311688311688
		VGG19: 0.047353896103896106
		VGG16: 0.04623376623376623
		ResNet50: 0.10047077922077922
		ResNet101V2: 0.0873538961038961
		Xception: 0.07337662337662337
		MobileNetV3Large: 0.1391396103896104
		EfficientNetV2B0: 0.09573051948051949
	DISGUST:
		SilNet: 0.15103896103896103
		WeiNet: 0.1782792207792208
		AlexNet: 0.04391233766233766
		SongNet: 0.09646103896103896
		InceptionV3: 0.22342532467532467
		VGG19: 0.1902435064935065
		VGG16: 0.18667207792207793
		ResNet50: 0.18681818181818183
		ResNet101V2: 0.1901948051948052
		Xception: 0.15433441558441557
		MobileNetV3Large: 0.2061525974025974
		EfficientNetV2B0: 0.1837012987012